## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-11-22-42-35 +0000,bbc,Moment newly opened bridge partially collapses...,https://www.bbc.com/news/videos/c231yry75m3o?a...
1,2025-11-11-22-39-00 +0000,wsj,Opinion | A 50-Year Mortgage Is a Bad Deal,https://www.wsj.com/opinion/50-year-mortgage-d...
2,2025-11-11-22-36-55 +0000,nypost,Florida parents arrested after 4-year-old twin...,https://nypost.com/2025/11/11/us-news/florida-...
3,2025-11-11-22-36-06 +0000,nyt,Pakistan ‘in a State of War’ After Explosion K...,https://www.nytimes.com/2025/11/11/world/asia/...
4,2025-11-11-22-34-12 +0000,wapo,U.S. aircraft carrier nears Latin America as V...,https://www.washingtonpost.com/national-securi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2356/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
63,trump,36
108,shutdown,28
126,new,18
107,government,17
124,market,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
291,2025-11-11-01-13-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...,130
14,2025-11-11-22-13-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...,109
267,2025-11-11-03-12-00 +0000,wsj,The decision by eight members of the Senate De...,https://www.wsj.com/politics/policy/democrats-...,101
138,2025-11-11-17-01-00 +0000,wsj,The Republican-led Senate late Monday passed a...,https://www.wsj.com/politics/policy/senate-pas...,96
36,2025-11-11-21-15-13 +0000,nyt,Trump Lauds ‘Very Big’ Shutdown Victory for G....,https://www.nytimes.com/2025/11/11/us/politics...,88


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
291,130,2025-11-11-01-13-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...
14,64,2025-11-11-22-13-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...
87,52,2025-11-11-19-22-21 +0000,nyt,India’s Prime Minister Vows Justice After New ...,https://www.nytimes.com/2025/11/11/world/asia/...
246,44,2025-11-11-08-43-05 +0000,wsj,"Stock Market Today: Dow Jumps, Nasdaq Slips; A...",https://www.wsj.com/livecoverage/stock-market-...
61,43,2025-11-11-20-02-06 +0000,nypost,Alleged drunken NJ driver who killed hockey st...,https://nypost.com/2025/11/11/us-news/alleged-...
75,43,2025-11-11-19-40-01 +0000,nypost,"Long Island grandma, 83, ‘had no idea’ she was...",https://nypost.com/2025/11/11/us-news/li-grand...
295,39,2025-11-11-00-52-53 +0000,nyt,Syria’s President Meets Trump at White House f...,https://www.nytimes.com/2025/11/10/world/middl...
42,35,2025-11-11-20-58-15 +0000,nypost,Anti-parade Zohran Mamdani touts affordable ho...,https://nypost.com/2025/11/11/us-news/anti-par...
156,32,2025-11-11-16-17-04 +0000,nypost,Auto parts CEO tells drivers to ‘stock up’ bef...,https://nypost.com/2025/11/11/business/auto-pa...
2,31,2025-11-11-22-36-55 +0000,nypost,Florida parents arrested after 4-year-old twin...,https://nypost.com/2025/11/11/us-news/florida-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
